# Transform custom CRS to WGS84

This notebook uses Geopandas to import a Shapefile, transform the CRS from a projected CRS to unprojected WGS84, and write the new files to disk

In [1]:
# import required libraries
import geopandas as gpd

In [2]:
# read the Shapefiles into memory
df_stormwater = gpd.read_file('./data/stormwater')

In [3]:
# examine first 10
df_stormwater.head(10)

,objectid,id,status,dia,type,box_size,flow_dir,rim_elev,inv_elev,invert,connection,geometry
0,1,SDMH C.08,ASBUILT,4.0,None,None,SW,5643.42,5629.46,13.96,2.0,POINT (3131894.760 1607146.735)
1,2,SDMH C45,ASBUILT,5.0,None,None,N,5621.11,5610.01,11.10,2.0,POINT (3130748.091 1609263.615)
2,3,SDMH G44-1,ASBUILT,5.0,None,None,NW,5651.02,5640.99,10.03,3.0,POINT (3129356.405 1607942.718)
3,4,MH A8,DESIGN,4.0,BOX BASE,86 x 86,NE,5642.91,5630.23,12.68,3.0,POINT (3121457.430 1601359.175)
4,5,MH A2,DESIGN,4.0,BOX BASE,107 x 107,NE,5613.21,5596.76,16.45,2.0,POINT (3121168.810 1602034.181)
5,6,SDMH B33,ASBUILT,5.0,None,None,NE,5610.69,5600.92,9.77,3.0,POINT (3131799.136 1609425.596)
6,7,SDMH G40-3,ASBUILT,5.0,None,None,W,5648.68,5641.56,7.12,2.0,POINT (3130037.979 1608275.515)
7,8,SDMH G49-6,ASBUILT,5.0,None,None,N,5680.13,5672.13,8.00,2.0,POINT (3128138.876 1607408.414)
8,9,MH F13,DESIGN,6.0,None,None,NW,5707.87,5697.23,10.64,4.0,POINT (3128296.332 1605782.683)
9,10,SDMH G44-7.1,ASBUILT,5.0,None,None,SE,5654.55,5644.82,9.73,3.0,POINT (3129203.788 1608097.663)


In [4]:
df_stormwater.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   objectid    387 non-null    int64   
 1   id          350 non-null    object  
 2   status      378 non-null    object  
 3   dia         342 non-null    float64 
 4   type        80 non-null     object  
 5   box_size    78 non-null     object  
 6   flow_dir    350 non-null    object  
 7   rim_elev    350 non-null    float64 
 8   inv_elev    348 non-null    float64 
 9   invert      344 non-null    float64 
 10  connection  350 non-null    float64 
 11  geometry    387 non-null    geometry
dtypes: float64(5), geometry(1), int64(1), object(5)
memory usage: 36.4+ KB


In [5]:
# no CRS assigned to data on import, despite .prj file
print(df_stormwater.crs)

None


In [6]:
# open the .prj file and store reference to the WKT definition
with open('./data/stormwater/stormwater.prj') as f:
    wkt_def = f.read()

In [7]:
# definition now stored as string
wkt_def

'PROJCS["SterlingRanchV12",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2999959.5576162],PARAMETER["False_Northing",999809.5290959987],PARAMETER["Central_Meridian",-105.5],PARAMETER["Standard_Parallel_1",38.45],PARAMETER["Standard_Parallel_2",39.75],PARAMETER["Scale_Factor",1.0003150391],PARAMETER["Latitude_Of_Origin",37.8333333333],UNIT["Foot_US",0.3048006096012192]]'

In [8]:
# first assign this definition to the dataset
df_stormwater.crs = wkt_def
print(df_stormwater.crs)

PROJCS["SterlingRanchV12",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2999959.5576162],PARAMETER["False_Northing",999809.5290959987],PARAMETER["Central_Meridian",-105.5],PARAMETER["Standard_Parallel_1",38.45],PARAMETER["Standard_Parallel_2",39.75],PARAMETER["Scale_Factor",1.0003150391],PARAMETER["Latitude_Of_Origin",37.8333333333],UNIT["Foot_US",0.3048006096012192]]


In [9]:
# then use the original CRS def to convert to desired
df_stormwater_wgs84 = df_stormwater.to_crs("EPSG:4326")

In [10]:
# verify transformation
df_stormwater_wgs84.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [11]:
# verify geoms are now in lon/lat
df_stormwater_wgs84['geometry'].sample(10)

314    POINT (-105.07121 39.48297)
218    POINT (-105.06916 39.48347)
211    POINT (-105.03209 39.49942)
271    POINT (-105.06927 39.47922)
153    POINT (-105.03659 39.50288)
17     POINT (-105.03346 39.49823)
357    POINT (-105.03204 39.50300)
190    POINT (-105.03569 39.49978)
179    POINT (-105.06979 39.47895)
144    POINT (-105.07218 39.48525)
Name: geometry, dtype: geometry

In [12]:
# write data to GeoJSON
df_stormwater_wgs84.to_file('./data/stormwater.json', driver='GeoJSON')

In [13]:
# write data to new Shapefile (within its own directory)
import os
os.mkdir('./data/stormwater_wgs84')
df_stormwater_wgs84.to_file('./data/stormwater_wgs84/stormwater_wgs84.shp')